In [1]:
# import libraries
import os
import subprocess
import shutil
import fnmatch
from osgeo import gdal
import cv2

In [58]:
# function that splits Geotiffs into required pieces

def create_tiles(src_path, dst_path):

    # specify tile dimension
    tile_size_x = 512
    tile_size_y = 512

    # dynamically get the image dimension
    dataset = gdal.Open(src_path)
    band = dataset.GetRasterBand(1) #1
    xsize = band.XSize
    ysize = band.YSize

    # clip image using using tile_size and gdal_translate iteratively
    for i in range(0, xsize, tile_size_x):
        for j in range(0, ysize, tile_size_y):
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i) + ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(src_path) + " " + str(dst_path) + "_" + str(i) + "_" + str(j) + ".tif" #.tif
            os.system(com_string)

In [48]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [62]:
source_path = r"/content/drive/MyDrive/gtif_to_png"
dest_path = r"/content/drive/MyDrive/gtf" # needs to exist
sen_path = r"/content/drive/MyDrive/gtf"

# splits all geotiff in input path folder
def gtiff_split(src_path, dest_path):
    for tile in glob.glob(f'{DATA_PATH}/Tile *'):
      for pla in glob.glob(f'{tile}/Planet_*_FullScene/*.tif'):
        dest_path1 = os.path.join(dest_path, pla[:-4])
        os.mkdir(dest_path1)      
        create_tiles(os.path.join(source_path, pla), os.path.join(dest_path1, pla[:-4]))
      for sen in glob.glob(f'{tile}/Sentinel_*/*.tif'):
        sen_path1 = os.path.join(sen_path, sen[:-4])
        os.mkdir(sen_path1)      
        create_tiles(os.path.join(source_path, sen), os.path.join(sen_path1, sen[:-4]))

In [ ]:
# ## 1 folder
#       for file in os.listdir(src_path): # all tifs in one dir
#         # print(file[:-4])
#         dest_path1 = os.path.join(dest_path, file[:-4])
#         os.mkdir(dest_path1)      
#         create_tiles(os.path.join(source_path, file), os.path.join(dest_path1, file[:-4]))
#     print(f"success! all files have been split")

In [63]:
gtiff_split(source_path, dest_path)

20170929_145600-01_Mosaic_clip
20181014_152754_0f3b_3B_AnalyticMS_SR_clip
20181004_151059-100_Mosaic_clip
20180711_150702_1044_3B_AnalyticMS_SR_clip
20190302_162738_89_106f_3B_AnalyticMS_SR_clip
20200919_133726_1054_3B_AnalyticMS_SR_clip
20201122_132436-37_Mosaic_clip
20210213_150034-33_Mosaic_clip
success! all files have been split


In [64]:
# function converts geotiff to png files (8bit)
def geotiff_to_png(input_tiff_img, output_png_img, output_pix_type='Byte', output_format='png', compress='png', percentiles=[2, 98]):

    img_src = gdal.Open(input_tiff_img)

    cmd = ['gdal_translate', '-ot', output_pix_type, '-of', output_format, '-co', compress]

    for band_id in range(img_src.RasterCount):
        band_id = band_id + 1
        band = img_src.GetRasterBand(band_id)

        # cal the band minimum and max values of raster
        b_min = band.GetMinimum()
        b_max = band.GetMaximum()

        # if not exist minimum and maximum values
        if b_min is None or b_max is None:
            (b_min, b_max) = band.ComputeRasterMinMax(1)
            if b_min == b_max:
                b_min -= 1

        cmd.append('-scale_{}'.format(band_id))
        cmd.append('{}'.format(b_min))
        cmd.append('{}'.format(b_max))
        cmd.append('{}'.format(0))
        cmd.append('{}'.format(255))

    cmd.append(input_tiff_img)
    cmd.append(output_png_img)
    print("Conversion command:", cmd)
    subprocess.call(cmd)

In [71]:
# function that activates the converter wrt geotiff in input path
def converter(input_path, output_path):
    # TODO: automate so its able to read all dirs in gtf
    # convert each file to output
    for file in os.listdir(input_path):
      # TODO: mkdir new png_folder
        geotiff_to_png(os.path.join(input_path, file), (output_path + file[:-4] + ".png"))
    print(f"success! all files have been converted to png")

In [72]:
# move xml extension files from png_path to its xml_path
def separate_png_xml(src_path, dst_path, ext):
    for file in fnmatch.filter(os.listdir(src_path), ext):
        shutil.move(os.path.join(src_path, file), os.path.join(dst_path, file))
    print(f"success!! xml file has been moved to Palm_xml")

In [73]:
# directories
geotiff_path = r"/content/drive/MyDrive/gtf/20170929_145600-01_Mosaic_clip"
png_path = r"/content/drive/MyDrive/gtf/"
xml_path = " "

In [74]:
# Convert the geotiffs to png
converter(input_path=geotiff_path, output_path=png_path)

Conversion command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-co', 'png', '-scale_1', '37.0', '453.0', '0', '255', '-scale_2', '140.0', '629.0', '0', '255', '-scale_3', '109.0', '572.0', '0', '255', '-scale_4', '893.0', '5871.0', '0', '255', '/content/drive/MyDrive/gtf/20170929_145600-01_Mosaic_clip/20170929_145600-01_Mosaic_clip_0_0.tif', '/content/drive/MyDrive/gtf/20170929_145600-01_Mosaic_clip_0_0.png']
Conversion command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-co', 'png', '-scale_1', '31.0', '405.0', '0', '255', '-scale_2', '139.0', '517.0', '0', '255', '-scale_3', '118.0', '562.0', '0', '255', '-scale_4', '1204.0', '4412.0', '0', '255', '/content/drive/MyDrive/gtf/20170929_145600-01_Mosaic_clip/20170929_145600-01_Mosaic_clip_0_512.tif', '/content/drive/MyDrive/gtf/20170929_145600-01_Mosaic_clip_0_512.png']
Conversion command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-co', 'png', '-scale_1', '29.0', '562.0', '0', '255', '-scale_2', '132.0', '704.0', '0',

In [ ]:
# Remove the all xml files from the png directory to xml folder
separate_png_xml(src_path=png_path, dst_path=xml_path, ext='*.xml')

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import glob
import pandas as pd
import torch

from torch.utils.data import Dataset
from rasterio.plot import show

In [ ]:
DATA_PATH = 'data'
data_map = []
pla_map = []

for tile in glob.glob(f'{DATA_PATH}/Tile *'):
  shape_files = glob.glob(f'{tile}/*.shp')
  if len(shape_files) == 0:
    continue

  for planet_img in glob.glob(f'{tile}/Planet_*_FullScene/*.tif'):
    data_map.extend([planet_img, shape_files[0]])
    pla_map.append(planet_img)
  
#   for sentinel_img in glob.glob(f'{tile}/Sentinel_*/*.tif'):
#     data_map.extend([sentinel_img, shape_files[0]])

# print(data_map[1::2])

df = pd.DataFrame({'img_path': data_map[::2], 'shape_path': data_map[1::2]})
#df.shape
#len(data_map)
#print(pla_map)

In [ ]:
PNG_PATH = 'data/pngs/'
def generate_pngs():
    
    for idx in df.index:
        print(idx)
#        print(data_map[idx])
        tiff_name = pla_map[idx]
        png_save_name = tiff_name.split('\\')[-1].replace('.tif', '.png')
        print(png_save_name)
#        break
        geo_img = rasterio.open(df.iloc[idx, 0]) #loop thru length of dataframe instead and pass idx 
    
        print(geo_img)
        image = geo_img.read()

        geo_points = gpd.read_file(df.iloc[idx, 1])
        print(geo_points)
        mask = create_mask(geo_points, geo_img)
        
        mask = Image.fromarray(mask*255) # for visualization
        # print(PNG_PATH + png_save_name) #For model - imread slice last channel
        mask = mask.convert('RGB')
        mask.save(PNG_PATH + png_save_name)

#     return image, mask

generate_pngs()